In [3]:
!pip install transformers

     |████████████████████████████████| 1.4MB 7.8MB/s 
     |████████████████████████████████| 890kB 29.6MB/s 
     |████████████████████████████████| 2.9MB 47.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=2b97fe5528f249434528532317d642a3ac294d6bbcbf56e0acc10e13b82492cd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [24]:
from pathlib import Path

,text,label
0,What could be more schlocky than the idea of p...,1
1,"""Film noir"" is an overused expression when it ...",1
2,I don't know how or why this film has a meager...,1
3,I used to watch this show when I was a little ...,1
4,Its no surprise that Busey later developed a t...,1


In [10]:
#!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xfz aclImdb_v1.tar.gz

In [4]:
from transformers import AutoTokenizer, AutoModel
import unicodedata
import torch
from transformers import *
import pandas as pd
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from torch import cuda

from sklearn.svm import OneClassSVM
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import metrics

import json


In [104]:
class BertEmbedding(torch.nn.Module):
    def __init__( self ):
        super(BertEmbedding, self).__init__()
        self.l1 = AutoModel.from_pretrained("bert-base-uncased",  output_hidden_states=True)

    def forward(self, ids,mask, token_type_ids):
        out_0, output_1, hidden_states = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        embd = torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1).max(dim=1)[0]
        return embd
    

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained("bert-base-uncased")
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 1)
    
    def forward(self, ids, mask, token_type_ids):
        output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        
        output_2 = self.l2(output_1['pooler_output'])
        output = self.l3(output_2)
        return output


In [105]:
device = 'cuda' if cuda.is_available() else 'cpu'
model = BERTClass()
_ = model.to(device)
_ = model.eval()


In [106]:
class TextDataset(Dataset):
    """
      Text dataset loader: Load data and feed 'em to BERT 

    """
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            truncation=True,
            padding = "max_length"
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [107]:
pos_files = Path("/content/aclImdb/train/pos").rglob("*.txt")
#len( list(pos_files))

neg_files = Path("/content/aclImdb/train/neg").rglob("*.txt")
#len( list(neg_files))

data = [] 
for file in pos_files:
  data.append( ( open(file).read(), 1))

for file in neg_files:
  data.append( ( open(file).read(), 0))
train_dataset = pd.DataFrame( data=data, columns = ["text", "label"])



data = [] 
pos_files = Path("/content/aclImdb/test/pos").rglob("*.txt")
#len( list(pos_files))

neg_files = Path("/content/aclImdb/test/neg").rglob("*.txt")
#len( list(neg_files))

data = [] 
for file in pos_files:
  data.append( ( open(file).read(), 1))

for file in neg_files:
  data.append( ( open(file).read(), 0))
test_dataset = pd.DataFrame( data=data, columns = ["text", "label"])





In [91]:
training_set[0]

{'ids': tensor([  101,  2054,  2071,  2022,  2062,  8040,  7317,  7432,  2100,  2084,
          1996,  2801,  1997,  2797, 18145,  2893,  2920,  2007,  1996,  2308,
          2027,  1005,  2128,  4011,  2000,  2022, 22624,  2006,  1029,  1998,
          2087,  1997,  1996,  7982,  2004,  2517,  2003,  6669,  7221,  2389,
          1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  2021,
          1996,  5889,  2735,  1996, 13764,  8649,  2046,  2242,  2008,  3084,
          3168,  1012,  2017,  2064,  2156,  2613,  2111,  2369,  1996,  4895,
         22852,  3210,  1012,  1998,  1996,  9855,  2003,  6919,  1012,  2169,
          3496,  2515,  2074,  2054,  2009,  2038,  2000,  1998,  4515,  2302,
         11920,  2006,  2205,  2146,  1012,  1026,  7987,  1013,  1028,   102]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1

In [108]:
MAX_LEN = 100
training_set = TextDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = TextDataset(test_dataset, tokenizer, MAX_LEN)

In [109]:
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [110]:
next(iter(training_loader))

{'ids': tensor([[  101,  3398,  1012,  ..., 20773,  4009,   102],
         [  101,  1045,  3728,  ...,  3185,  1998,   102],
         [  101,  2202,  1000,  ...,  2008,  2038,   102],
         ...,
         [  101,  2195,  2086,  ..., 11615,  7330,   102],
         [  101,  2017,  2064,  ...,  2143,  2000,   102],
         [  101,  1037,  2843,  ..., 23277,  5654,   102]]),
 'mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 'targets': tensor([0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]])}

In [111]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()

        targets = targets.reshape( targets.shape[0], 1 )
        outputs = outputs.reshape( targets.shape[0], 1 )
        loss = criterion(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets


In [112]:
criterion = torch.nn.BCEWithLogitsLoss()
 

In [113]:
LEARNING_RATE = 1e-05
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE) 

data = next(iter(training_loader))
ids = data['ids'].to(device, dtype = torch.long)
mask = data['mask'].to(device, dtype = torch.long)
token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
targets = data['targets'].to(device, dtype = torch.float)


m = AutoModel.from_pretrained("bert-base-uncased").to(device)

outputs = model(ids, mask, token_type_ids)
#
#targets = targets.reshape( targets.shape[0] )
#outputs = outputs.reshape( outputs.shape[0] )

#outputs['pooler_output'].shape

torch.Size([16, 768])

In [114]:
EPOCHS = 2
for epoch in range(EPOCHS):
    train(epoch)


    outputs, targets = validation(testing_loader)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

Epoch: 0, Loss:  0.7443519830703735
Accuracy Score = 0.86444
F1 Score (Micro) = 0.86444
F1 Score (Macro) = 0.8642257919647056
Epoch: 1, Loss:  0.1843758225440979
Accuracy Score = 0.86796
F1 Score (Micro) = 0.86796
F1 Score (Macro) = 0.8677394018076157
